In [12]:
# Link the google drive in it
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#from xgboost import XGBClassifier
#import numpy as np
#import pandas as pd
#import sklearn
#from sklearn.multiclass import OneVsRestClassifier

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from pathlib import Path
from pprint import pprint

In [38]:
# Direct to the file location in the Google Drive
datapath = Path("/content/drive/MyDrive/0_2024_Mine_PreProject/2025_VimMatch_Clients Recommendation AI Agent/Vimmax/Recommendation System_XGBoost_info")
# Load CSVs
clients = pd.read_csv(datapath / "sample_clients.csv")
products = pd.read_csv(datapath / "sample_products.csv")
interactions = pd.read_csv(datapath / "sample_interactions.csv")

# (failed) Create a folder in the root directory
#!mkdir -p "/content/drive/MyDrive/0_2024_Mine_PreProject/2025_VimMatch_Clients Recommendation AI Agent/Vimmax/Recommendation System_XGBoost_info/Test") #"/content/drive/My Drive/My Folder"

/bin/bash: -c: line 1: syntax error near unexpected token `)'
/bin/bash: -c: line 1: `mkdir -p "/content/drive/MyDrive/0_2024_Mine_PreProject/2025_VimMatch_Clients Recommendation AI Agent/Vimmax/Recommendation System_XGBoost_info/Test") #"/content/drive/My Drive/My Folder"'


In [35]:
"""
# Load your CSV files
clients = pd.read_csv("sample_clients.csv")
products = pd.read_csv("sample_products.csv")
interactions = pd.read_csv("sample_interactions.csv")
"""
# Merge into one dataset
df = interactions.merge(clients, on="client_id").merge(products, on="product_id")

# Label encode outcome
df["label"] = df["outcome"].map({"Partnered": 1, "Rejected": 0})

# Select columns to use
categorical = [
    "platform", "region_x", "content_type", "price_tier",
    "price_range", "aesthetic", "target_user", "region_y"
]
numeric = ["audience_size", "avg_engagement"]

# Encode categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[categorical])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical))

# Combine into final training dataset
X = pd.concat([df[numeric].reset_index(drop=True), encoded_df], axis=1)
y = df["label"]

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train XGBoost model
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Predict scores on full dataset (for exploration/recommendation)
df["score"] = model.predict_proba(X)[:, 1]
df.to_csv("match_recommendations.csv", index=False)
print("✅ Saved: match_recommendations.csv")


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

✅ Saved: match_recommendations.csv


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:45:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels w

## Below is the old script, try it for learning

In [3]:
# Direct to the file location in the Google Drive
datapath = Path("/content/drive/MyDrive/0_2024_Mine_PreProject/2025_VimMatch_Clients Recommendation AI Agent/Vimmax/Recommendation System_XGBoost_info")
# Load CSVs
clients = pd.read_csv(datapath / "sample_clients.csv")
products = pd.read_csv(datapath / "sample_products.csv")
interactions = pd.read_csv(datapath / "sample_interactions.csv")

In [20]:
# Merge all
df = interactions.merge(clients, on="client_id").merge(products, on="product_id")

#print(df)

# Label encode outcome
df["label"] = df["outcome"].map({"Partnered": 1, "Rejected": 0})
pprint(df)
df.shape

   client_id  product_id    outcome              name   platform  \
0        101         201  Partnered         Chef Emma    YouTube   
1        102         201   Rejected  GrillMaster Mike  Instagram   
2        103         202  Partnered       BakingQueen     TikTok   

   audience_size region_x content_type  avg_engagement price_tier  \
0         500000       US       Baking             3.2        Mid   
1         300000       UK     Grilling             2.5       High   
2         800000       US       Baking             4.1        Mid   

  previous_collabs            line_name price_range   aesthetic  \
0  OurPlace, Ninja  Signature Cast Iron        High      Rustic   
1            Weber  Signature Cast Iron        High      Rustic   
2       KitchenAid         Eco Utensils         Low  Minimalist   

            target_user region_y launch_date  label  
0            Home Cooks       US  2024-06-01      1  
1            Home Cooks       US  2024-06-01      0  
2  Eco-conscious bu

(3, 18)

In [21]:
# Feature selection
categorical = [
    "platform", "region_x", "content_type", "price_tier",
    "price_range", "aesthetic", "target_user", "region_y"
]
numeric = ["audience_size", "avg_engagement"]

print(type(categorical))
print(type(numeric))

# Transform the feature into lables

<class 'list'>
<class 'list'>


In [26]:
# One-hot encode categorical
encoder = OneHotEncoder(cFalse)
encoded = encoder.fit_transform(df[categorical])
#print(encoded)
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical))

In [27]:
# Combine features
X = pd.concat([df[numeric].reset_index(drop=True), encoded_df], axis=1)
y = df["label"]

In [28]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
# Train XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:41:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [30]:
# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [34]:
# Merge all
df = interactions.merge(clients, on="client_id").merge(products, on="product_id")

# Label encode outcome
df["label"] = df["outcome"].map({"Partnered": 1, "Rejected": 0})

# Feature selection
categorical = [
    "platform", "region_x", "content_type", "price_tier",
    "price_range", "aesthetic", "target_user", "region_y"
]
numeric = ["audience_size", "avg_engagement"]

# One-hot encode categorical
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[categorical])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical))

# Combine features
X = pd.concat([df[numeric].reset_index(drop=True), encoded_df], axis=1)
y = df["label"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:44:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels w